In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NCC\\Documents\\Projects\\solar-radiation-forecasting-with-xgboost\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\NCC\\Documents\\Projects\\solar-radiation-forecasting-with-xgboost'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.solar_radiation_prediction.constants import *
from src.solar_radiation_prediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.solar_radiation_prediction import logger
from src.solar_radiation_prediction.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Successfully downloaded file: {filename} with the following info: {headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} with filesize of: {get_size(self.config.local_data_file)}")
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extracts the zip file to the specified data directory
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# updating pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    # Check if 'root_dir' attribute is defined in the configuration file
    if 'root_dir' not in data_ingestion_config:
        raise ValueError("Configuration file does not contain 'root_dir' attribute.")
    else:
        raise e


[2024-05-13 09:14:49,553: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-13 09:14:49,556: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-13 09:14:49,559: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-13 09:14:49,561: INFO: common: directory created at: artifacts]
[2024-05-13 09:14:49,563: INFO: common: directory created at: artifacts/data_ingestion]
[2024-05-13 09:15:37,283: INFO: 4119466327: Successfully downloaded file: artifacts/data_ingestion/data.zip with the following info: Connection: close
Content-Length: 2598964
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5ed5175120085ab1ccd8e6663cef2db002202adf05912798a889541ac6b1e144"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 47C8:5762F:DC99E:181C9A:6641CBFD
Accept-Rang